## Datasets processing

In [10]:
from pathlib import Path
import sys
 
if sys.argv:
    sys.path.insert(0, str(Path('/Users/ogrobertino/UpliftModelingResearch/').resolve()))
%load_ext autoreload
%autoreload 2
import pandas as pd
from src.distributions_check import check_feature_distributions_by_stat_test, check_feature_distributions_by_model, check_propensity
from src.datasets import sample_features, TorchDataset, NumpyDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


TLDR:
1. Ориентируюсь на LAZADA, хотя там распределения на трейне и тесте отличаются
2. Как-нибудь собрать на данных яндекса анонимный датасет
3. Поискать фича инжиниринг по x5, сделать там фичи. Но заеб в том что никакую метрику кроме uplift@0.3 мы не получим
4. Criteo хорош так как есть бенчмарк, распределение одинаковое (так как нет теста изначально), но очень мало фичей

### Loading datasets from internet

#### Criteo

https://huggingface.co/datasets/criteo/criteo-uplift https://www.uplift-modeling.com/en/latest/api/datasets/fetch_criteo.html

In [19]:
df = pd.read_csv("hf://datasets/criteo/criteo-uplift/criteo-research-uplift-v2.1.csv.gz")

In [20]:
print('Event-rate treatment: ' + str(df[df['treatment'] == 1]['conversion'].mean()),\
      '\nEvent-rate control: ' + str(df[df['treatment'] == 0]['conversion'].mean()))
# print(df[df['treatment'] == 1]['exposure'].mean(), df[df['treatment'] == 0]['exposure'].mean())
# print(df[df['treatment'] == 1]['visit'].mean(), df[df['treatment'] == 0]['visit'].mean())

Event-rate train: 0.0030894610674129645 
Event-rate test: 0.0019375880152813366


In [21]:
from sklearn.model_selection import train_test_split
count_for_test = int(0.25 * len(df))
train_data, test_data = train_test_split(df, test_size=count_for_test)

In [22]:
check_feature_distributions_by_stat_test(train_data, test_data, plot=False, print_=False)


Распределения похожи для всех фичей (p-value >= 0.05).


In [23]:
check_feature_distributions_by_model(train_data, test_data)

AUC: 0.5002
Тренировочные и тестовые датасеты РАЗЛИЧАЮТСЯ!


In [24]:
train_data['target'] = train_data['conversion']
train_data = train_data.drop(['conversion', 'visit', 'exposure'], axis=1)
test_data['target'] = test_data['conversion']
test_data = test_data.drop(['conversion', 'visit', 'exposure'], axis=1)

In [29]:
percents = [100, 90, 80, 70, 60, 50]
sample_features(percents, train_data, test_data, 'data/criteo')

#### Lazada

https://github.com/kailiang-zhong/DESCN/tree/main/data/Lazada_dataset

In [4]:
# from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [5]:
train_data = pd.read_csv('../data/trash/lazada/full_trainset.csv')
test_data = pd.read_csv('../data/trash/lazada/full_testset.csv')

In [6]:
train_data['target'] = train_data['label']
train_data['treatment'] = train_data['is_treat']
test_data['target'] = test_data['label']
test_data['treatment'] = test_data['is_treat']

In [7]:
print('TRAIN')
print('Event-rate treatment: ' + str(train_data[train_data['treatment'] == 1]['target'].mean()),\
      '\nEvent-rate control: ' + str(train_data[train_data['treatment'] == 0]['target'].mean()))

print('TEST')
print('Event-rate treatment: ' + str(test_data[test_data['treatment'] == 1]['target'].mean()),\
      '\nEvent-rate control: ' + str(test_data[test_data['treatment'] == 0]['target'].mean()))


TRAIN
Event-rate treatment: 0.056638115631691646 
Event-rate control: 0.009442739697915526
TEST
Event-rate treatment: 0.036997528569316235 
Event-rate control: 0.03325784312598434


In [8]:
train_data = train_data.drop(['data_id', 'label', 'is_treat'],axis=1)
test_data = test_data.drop(['data_id', 'label', 'is_treat'],axis=1)

In [66]:
# import warnings
# warnings.filterwarnings('ignore')
# # scaler = MinMaxScaler(feature_range=(0, 1))
# scaler = StandardScaler()
# cols = list(set(train_data.columns) - set(['target', 'treatment']))
# train_data.loc[:, cols] = scaler.fit_transform(train_data.loc[:, cols])
# test_data.loc[:, cols] = scaler.transform(test_data.loc[:, cols])

In [12]:
check_propensity(train_data, test_data)

AUC train: 0.9356
AUC test: 0.5025


In [69]:
check_feature_distributions_by_stat_test(train_data, test_data, plot=False, print_=False)


Обнаружены значимые различия в распределениях для следующих фичей:
Фича 1, Фича 2, Фича 3, Фича 4, Фича 5, Фича 6, Фича 7, Фича 8, Фича 9, Фича 10, Фича 11, Фича 12, Фича 13, Фича 14, Фича 15, Фича 16, Фича 17, Фича 18, Фича 19, Фича 20, Фича 21, Фича 22, Фича 23, Фича 24, Фича 25, Фича 26, Фича 27, Фича 28, Фича 29, Фича 30, Фича 31, Фича 32, Фича 33, Фича 34, Фича 35, Фича 36, Фича 37, Фича 38, Фича 39, Фича 40, Фича 41, Фича 42, Фича 43, Фича 44, Фича 45, Фича 46, Фича 47, Фича 53, Фича 54, Фича 58, Фича 59, Фича 60, Фича 61, Фича 62, Фича 63, Фича 64, Фича 66, Фича 67, Фича 68, Фича 69, Фича 70, Фича 72, Фича 73, Фича 74, Фича 75, Фича 76, Фича 77, Фича 78, Фича 79, Фича 80, Фича 81, Фича 82, Фича 83, Фича 84, Фича 85


In [70]:
check_feature_distributions_by_model(train_data, test_data)

AUC: 0.7973
Тренировочные и тестовые датасеты РАЗЛИЧАЮТСЯ!


In [71]:
percents = [100, 50]
sample_features(percents, train_data, test_data, '../data/lazada2')

#### X5

https://ods.ai/competitions/x5-retailhero-uplift-modeling/data https://www.uplift-modeling.com/en/latest/api/datasets/fetch_x5.html#x5-retailhero-uplift-modeling-dataset

Пока оставлю: нужен фича инжениринг (впадлу пока); мало данных, тест приватный - надо засылать в контест и так смотреть, на данном этапе это лишняя работа; слабый тритмент эффект

In [96]:
df = pd.read_csv('data/trash/x5/data/uplift_train.csv')
len(df)

200039

In [95]:
print('Event-rate treatment: ' + str(df[df['treatment_flg'] == 1]['target'].mean()),\
      '\nEvent-rate control: ' + str(df[df['treatment_flg'] == 0]['target'].mean()))

Event-rate treatment: 0.6365109370780448 
Event-rate control: 0.6032800975434248


#### Lenta

https://www.youtube.com/watch?v=gIHwnDH9PD8 https://github.com/maks-sh/scikit-uplift/blob/master/notebooks/EDA_Lenta_dataset.ipynb

https://www.uplift-modeling.com/en/latest/api/datasets/fetch_lenta.html#lenta-uplift-modeling-dataset

Пока оставлю: не нравится что маленький эффект тритмента + не с чем сравнивать результаты. Топовое решение углублено в фича инжиниринг, мне это не надо

In [78]:
from sklift.datasets import fetch_lenta


dataset = fetch_lenta()
df, target, treatment = dataset.data, dataset.target, dataset.treatment

In [79]:
df['target'] = target
df['treatment'] = treatment
df.treatment = df.treatment.replace({'test': 1, 'control': 0})

/var/folders/2h/f3088zln2y36htvcq0wjvt94_1rc11/T/ipykernel_28376/2955555357.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.treatment = df.treatment.replace({'test': 1, 'control': 0})


In [82]:
print('Event-rate treatment: ' + str(df[df['treatment'] == 1]['target'].mean()),\
      '\nEvent-rate control: ' + str(df[df['treatment'] == 0]['target'].mean()))

Event-rate treatment: 0.11012576275654594 
Event-rate control: 0.1025786358297738


In [74]:
print('Event-rate treatment: ' + str(df[df['treatment'] == 1]['conversion'].mean()),\
      '\nEvent-rate control: ' + str(df[df['treatment'] == 0]['conversion'].mean()))

from sklearn.model_selection import train_test_split
count_for_test = int(0.25 * len(df))
train_data, test_data = train_test_split(df, test_size=count_for_test)
check_feature_distributions_by_stat_test(train_data, test_data, plot=False, print_=False)
check_feature_distributions_by_model(train_data, test_data)

Event-rate train: 0.11012576275654594 
Event-rate test: 0.1025786358297738

Распределения похожи для всех фичей (p-value >= 0.05).
AUC: 0.5002
Тренировочные и тестовые датасеты РАЗЛИЧАЮТСЯ!


In [ ]:
percents = [100, 90, 80, 70, 60, 50]
sample_features(percents, train_data, test_data, 'data/lenta')